In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%cd '/content/drive/My Drive/House-price-forecast'

/content/drive/My Drive/House-price-forecast


In [4]:
#基础
import numpy as np
import pandas as pd
import time

#绘图
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#模型
from sklearn.linear_model import Lasso, LassoCV, ElasticNet, ElasticNetCV, Ridge, RidgeCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from mlxtend.regressor import StackingCVRegressor
from sklearn.svm import SVR
import lightgbm as lgb
import xgboost as xgb

#模型相关
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error

#忽略警告
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')
print('The shape of training data:', train.shape)
print('The shape of testing data:', test.shape)

The shape of training data: (1458, 160)
The shape of testing data: (1459, 159)


In [6]:
train.isnull().sum().any()

False

In [7]:
y = np.log1p(train['SalePrice'])
train = train.drop('SalePrice', axis=1)

#检查训练集与测试集的维度是否一致
print('The shape of training data:', train.shape)
print('The length of y:', len(y))
print('The shape of testing data:', test.shape)

The shape of training data: (1458, 159)
The length of y: 1458
The shape of testing data: (1459, 159)


In [8]:
y.isnull().sum()

0

In [9]:
#采用十折交叉验证
n_folds = 10

def rmse_cv(model):
  kf = KFold(n_folds, shuffle=True, random_state=20)
  rmse = np.sqrt(-cross_val_score(model, train.values, y, scoring='neg_mean_squared_error', cv=kf))
  return(rmse)

def rmse(y, y_preds):
  return np.sqrt(mean_squared_error(y, y_preds))

### XGBoost调参

#### 1、最佳迭代次数：n_estimators

In [9]:
params = {'learning_rate': 0.01,
      'n_estimators': 6000, 
      'max_depth': 11,
      'subsample': 0.9,
      'colsample_bytree': 0.7,
      'min_child_weight': 1,
      'seed': 52,
      'gamma': 0,
      'reg_alpha': 0,
      'reg_lambda': 1}
cv_params = {'n_estimators': [200, 500, 800, 1000, 2000, 3000, 5000, 6000, 8000]}
xgbr = xgb.XGBRegressor(**params)
grid_xgbr = GridSearchCV(estimator=xgbr, param_grid=cv_params, scoring='neg_mean_squared_error', cv=10, verbose=1)
grid_xgbr.fit(np.array(train), np.array(y))
result = grid_xgbr.cv_results_
#print('每轮迭代运行结果:{}, {}'.format(result['mean_train_score'], result['std_train_score']))
print('参数的最佳取值：{}'.format(grid_xgbr.best_params_))
print('最佳模型得分:{}'.format(grid_xgbr.best_score_))

Fitting 10 folds for each of 9 candidates, totalling 90 fits
[07:15:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[07:15:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:15:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:15:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:15:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:15:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:15:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:15:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:15:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed: 35.9min finished


[07:51:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
参数的最佳取值：{'n_estimators': 3000}
最佳模型得分:-0.015104820300369842


In [10]:
result

{'mean_fit_time': array([ 0.80616078,  4.57343869,  9.2300158 , 12.52786803, 29.33173862,
        32.99010479, 38.52572479, 40.88207231, 46.12188926]),
 'mean_score_time': array([0.00146346, 0.00635142, 0.01647627, 0.0268548 , 0.09837255,
        0.11303117, 0.1313036 , 0.11626239, 0.12053096]),
 'mean_test_score': array([-2.45330175, -0.0243604 , -0.01545904, -0.01520463, -0.01510573,
        -0.01510482, -0.01510509, -0.01510523, -0.01510546]),
 'param_n_estimators': masked_array(data=[200, 500, 800, 1000, 2000, 3000, 5000, 6000, 8000],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'params': [{'n_estimators': 200},
  {'n_estimators': 500},
  {'n_estimators': 800},
  {'n_estimators': 1000},
  {'n_estimators': 2000},
  {'n_estimators': 3000},
  {'n_estimators': 5000},
  {'n_estimators': 6000},
  {'n_estimators': 8000}],
 'rank_test_score': array([9, 8, 7, 6, 5, 1, 2, 3, 4], dt

In [12]:
params = {'learning_rate': 0.01,
      'n_estimators': 3000, 
      'max_depth': 11,
      'subsample': 0.9,
      'colsample_bytree': 0.7,
      'min_child_weight': 1,
      'seed': 52,
      'gamma': 0,
      'reg_alpha': 0,
      'reg_lambda': 1}
cv_params = {'n_estimators': [2200, 2500, 2800, 3000, 3200, 3500, 3800, 4000, 4200, 4500, 4800]}
xgbr = xgb.XGBRegressor(**params)
grid_xgbr = GridSearchCV(estimator=xgbr, param_grid=cv_params, scoring='neg_mean_squared_error', cv=10, verbose=1)
grid_xgbr.fit(np.array(train), np.array(y))
result = grid_xgbr.cv_results_
#print('每轮迭代运行结果:{}, {}'.format(result['mean_train_score'], result['std_train_score']))
print('参数的最佳取值：{}'.format(grid_xgbr.best_params_))
print('最佳模型得分:{}'.format(grid_xgbr.best_score_))

Fitting 10 folds for each of 11 candidates, totalling 110 fits
[07:55:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[07:56:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:56:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:57:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:57:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:58:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:58:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:59:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:59:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

[Parallel(n_jobs=1)]: Done 110 out of 110 | elapsed: 62.2min finished


[08:57:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
参数的最佳取值：{'n_estimators': 3000}
最佳模型得分:-0.015104820300369842


In [14]:
result

{'mean_fit_time': array([29.73493893, 30.63010428, 31.72601666, 32.57875319, 33.29498451,
        34.45654204, 34.63806009, 35.04645514, 35.73538365, 36.83729208,
        37.36633852]),
 'mean_score_time': array([0.09084637, 0.09593482, 0.09277196, 0.09652143, 0.10464144,
        0.10375171, 0.10308754, 0.0891408 , 0.09101684, 0.09169021,
        0.09786186]),
 'mean_test_score': array([-0.01510513, -0.01510486, -0.01510486, -0.01510482, -0.01510489,
        -0.01510493, -0.01510501, -0.01510497, -0.01510493, -0.01510491,
        -0.01510509]),
 'param_n_estimators': masked_array(data=[2200, 2500, 2800, 3000, 3200, 3500, 3800, 4000, 4200,
                    4500, 4800],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_estimators': 2200},
  {'n_estimators': 2500},
  {'n_estimators': 2800},
  {'n_estimators': 3000},
  {'n_estimators': 3200},
  {'n_esti

#### 2、min_child_weight和max_depth

In [15]:
params = {'learning_rate': 0.01,
      'n_estimators': 3000, 
      'max_depth': 11,
      'subsample': 0.9,
      'colsample_bytree': 0.7,
      'min_child_weight': 1,
      'seed': 52,
      'gamma': 0,
      'reg_alpha': 0,
      'reg_lambda': 1}
cv_params = {'min_child_weight': [1, 2, 3, 4, 5, 6], 'max_depth': [5, 6, 7, 8, 9, 10, 11]}
xgbr = xgb.XGBRegressor(**params)
grid_xgbr = GridSearchCV(estimator=xgbr, param_grid=cv_params, scoring='neg_mean_squared_error', cv=10, verbose=1)
grid_xgbr.fit(np.array(train), np.array(y))
result = grid_xgbr.cv_results_
#print('每轮迭代运行结果:{}, {}'.format(result['mean_train_score'], result['std_train_score']))
print('参数的最佳取值：{}'.format(grid_xgbr.best_params_))
print('最佳模型得分:{}'.format(grid_xgbr.best_score_))

Fitting 10 folds for each of 42 candidates, totalling 420 fits
[11:24:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[11:24:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:24:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:25:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:25:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:25:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:26:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:26:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:26:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

[Parallel(n_jobs=1)]: Done 420 out of 420 | elapsed: 206.5min finished


[14:50:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
参数的最佳取值：{'max_depth': 5, 'min_child_weight': 3}
最佳模型得分:-0.013844205234061244


In [16]:
result

{'mean_fit_time': array([19.6026027 , 19.66422234, 19.54717247, 19.39716184, 19.88876374,
        19.33176496, 23.29881861, 23.20702763, 23.07030642, 22.89300139,
        22.79358137, 22.70243242, 27.15496264, 27.03648202, 26.71968372,
        26.55672479, 26.31905994, 26.25255034, 31.23664985, 30.91732395,
        30.68790443, 30.22166474, 29.98374367, 29.63154485, 33.11391943,
        33.82720792, 34.35954187, 33.85635808, 33.46703243, 33.16850238,
        32.31499701, 33.49989839, 35.87422695, 37.32481959, 36.92933478,
        36.82451563, 31.97521274, 33.33556056, 36.76831179, 40.02029898,
        40.57615101, 40.58199427]),
 'mean_score_time': array([0.03296959, 0.03303051, 0.03300784, 0.03343878, 0.04465194,
        0.03202214, 0.04586766, 0.04530866, 0.04477415, 0.04331725,
        0.04073465, 0.04079299, 0.06109149, 0.05962648, 0.05636654,
        0.05446839, 0.05306561, 0.05137095, 0.07439389, 0.07216885,
        0.07225838, 0.06598446, 0.06441405, 0.05943353, 0.08303468,
    

#### 3、gamma 

In [17]:
params = {'learning_rate': 0.01,
      'n_estimators': 3000, 
      'max_depth': 5,
      'subsample': 0.9,
      'colsample_bytree': 0.7,
      'min_child_weight': 3,
      'seed': 52,
      'gamma': 0,
      'reg_alpha': 0,
      'reg_lambda': 1}
cv_params = {'gamma': [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]}
xgbr = xgb.XGBRegressor(**params)
grid_xgbr = GridSearchCV(estimator=xgbr, param_grid=cv_params, scoring='neg_mean_squared_error', cv=10, verbose=1)
grid_xgbr.fit(np.array(train), np.array(y))
result = grid_xgbr.cv_results_
#print('每轮迭代运行结果:{}, {}'.format(result['mean_train_score'], result['std_train_score']))
print('参数的最佳取值：{}'.format(grid_xgbr.best_params_))
print('最佳模型得分:{}'.format(grid_xgbr.best_score_))

Fitting 10 folds for each of 7 candidates, totalling 70 fits
[14:56:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[14:56:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:56:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:57:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:57:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:57:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:58:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:58:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:58:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

[Parallel(n_jobs=1)]: Done  70 out of  70 | elapsed: 23.1min finished


[15:19:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
参数的最佳取值：{'gamma': 0}
最佳模型得分:-0.013844205234061244


#### 4、subsample和colsample_bytree

In [9]:
params = {'learning_rate': 0.01,
      'n_estimators': 3000, 
      'max_depth': 5,
      'subsample': 0.9,
      'colsample_bytree': 0.7,
      'min_child_weight': 3,
      'seed': 52,
      'gamma': 0,
      'reg_alpha': 0,
      'reg_lambda': 1}
cv_params = {'subsample': [0.6, 0.7, 0.8, 0.9, 1.0], 'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0]}
xgbr = xgb.XGBRegressor(**params)
grid_xgbr = GridSearchCV(estimator=xgbr, param_grid=cv_params, scoring='neg_mean_squared_error', cv=10, verbose=1)
grid_xgbr.fit(np.array(train), np.array(y))
result = grid_xgbr.cv_results_
#print('每轮迭代运行结果:{}, {}'.format(result['mean_train_score'], result['std_train_score']))
print('参数的最佳取值：{}'.format(grid_xgbr.best_params_))
print('最佳模型得分:{}'.format(grid_xgbr.best_score_))

Fitting 10 folds for each of 25 candidates, totalling 250 fits
[02:05:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[02:06:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:06:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:06:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:07:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:07:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:07:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:08:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:08:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed: 97.8min finished


[03:43:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
参数的最佳取值：{'colsample_bytree': 0.7, 'subsample': 0.6}
最佳模型得分:-0.01350188742592883


In [10]:
result

{'mean_fit_time': array([20.77341776, 19.65849676, 18.55563574, 17.5307189 , 15.91322839,
        23.47974966, 22.80694509, 21.12292247, 19.61606972, 18.290941  ,
        26.23824685, 24.92024889, 23.42352347, 21.8632555 , 19.91830678,
        29.01571393, 27.83774695, 25.96293855, 24.25251181, 22.00881391,
        32.06285958, 31.10121098, 28.98479025, 26.71920235, 23.89011576]),
 'mean_score_time': array([0.03220036, 0.03164058, 0.03258028, 0.03437645, 0.03120821,
        0.03374813, 0.03567753, 0.03360198, 0.03341243, 0.03327582,
        0.03566151, 0.03459668, 0.03432472, 0.03400421, 0.03252289,
        0.03415229, 0.03508239, 0.03476913, 0.03551581, 0.03358364,
        0.03500044, 0.03552749, 0.0356365 , 0.03645473, 0.03292286]),
 'mean_test_score': array([-0.01351611, -0.01352714, -0.01359658, -0.01371164, -0.01405421,
        -0.01350189, -0.01374173, -0.01377598, -0.01384421, -0.01418901,
        -0.0136482 , -0.01367606, -0.01375212, -0.01390798, -0.01433508,
        -0.013537

#### 5、reg_alpha和reg_lambda

In [11]:
params = {'learning_rate': 0.01,
      'n_estimators': 3000, 
      'max_depth': 5,
      'subsample': 0.6,
      'colsample_bytree': 0.7,
      'min_child_weight': 3,
      'seed': 52,
      'gamma': 0,
      'reg_alpha': 0,
      'reg_lambda': 1}
cv_params = {'reg_alpha': [0.05, 0.1, 0.5, 1, 2, 3], 'reg_lambda': [0.05, 0.1, 0.5, 1, 2, 3]}
xgbr = xgb.XGBRegressor(**params)
grid_xgbr = GridSearchCV(estimator=xgbr, param_grid=cv_params, scoring='neg_mean_squared_error', cv=10, verbose=1)
grid_xgbr.fit(np.array(train), np.array(y))
result = grid_xgbr.cv_results_
#print('每轮迭代运行结果:{}, {}'.format(result['mean_train_score'], result['std_train_score']))
print('参数的最佳取值：{}'.format(grid_xgbr.best_params_))
print('最佳模型得分:{}'.format(grid_xgbr.best_score_))

Fitting 10 folds for each of 36 candidates, totalling 360 fits
[03:47:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[03:47:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:47:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:48:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:48:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:49:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:49:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:50:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:50:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

[Parallel(n_jobs=1)]: Done 360 out of 360 | elapsed: 137.8min finished


[06:04:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
参数的最佳取值：{'reg_alpha': 0.05, 'reg_lambda': 1}
最佳模型得分:-0.013575151169877083


In [12]:
result

{'mean_fit_time': array([24.44246793, 24.32434533, 23.79427948, 23.84461319, 23.71915658,
        23.3632189 , 24.46509485, 24.37283423, 23.97710097, 23.83697555,
        24.72607501, 23.57180893, 23.88799355, 24.31556458, 23.88998697,
        23.57199731, 23.66110601, 23.46586192, 23.49158411, 23.5859081 ,
        23.78411326, 23.33546581, 23.15567186, 23.15948248, 22.12988596,
        22.30778565, 22.52216864, 22.63585901, 22.28099625, 22.5293606 ,
        19.91074243, 19.85986459, 20.00638669, 19.8292279 , 19.95238528,
        19.94295964]),
 'mean_score_time': array([0.03648353, 0.03702261, 0.03511884, 0.03568435, 0.03501341,
        0.0348393 , 0.03602324, 0.0365046 , 0.03504469, 0.03550222,
        0.03662097, 0.03529384, 0.0363363 , 0.03737619, 0.03567433,
        0.034794  , 0.03508124, 0.03500435, 0.03346391, 0.03448246,
        0.0344888 , 0.03321295, 0.03366249, 0.03257771, 0.03029318,
        0.03022802, 0.02962396, 0.03069394, 0.02948647, 0.03147342,
        0.02576511, 0.

#### 6、learning_rate

In [13]:
params = {'learning_rate': 0.01,
      'n_estimators': 3000, 
      'max_depth': 5,
      'subsample': 0.6,
      'colsample_bytree': 0.7,
      'min_child_weight': 3,
      'seed': 52,
      'gamma': 0,
      'reg_alpha': 0,
      'reg_lambda': 1}
cv_params = {'learning_rate': [0.005, 0.01, 0.02, 0.05, 0.08, 0.1]}
xgbr = xgb.XGBRegressor(**params)
grid_xgbr = GridSearchCV(estimator=xgbr, param_grid=cv_params, scoring='neg_mean_squared_error', cv=10, verbose=1)
grid_xgbr.fit(np.array(train), np.array(y))
result = grid_xgbr.cv_results_
#print('每轮迭代运行结果:{}, {}'.format(result['mean_train_score'], result['std_train_score']))
print('参数的最佳取值：{}'.format(grid_xgbr.best_params_))
print('最佳模型得分:{}'.format(grid_xgbr.best_score_))

Fitting 10 folds for each of 6 candidates, totalling 60 fits
[06:54:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[06:54:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:55:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:55:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:55:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:56:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:56:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:57:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:57:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed: 22.2min finished


[07:16:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
参数的最佳取值：{'learning_rate': 0.01}
最佳模型得分:-0.01350188742592883


In [14]:
result

{'mean_fit_time': array([22.64382863, 23.76768003, 24.46850123, 24.57234902, 19.76114542,
        17.77851887]),
 'mean_score_time': array([0.03292613, 0.03662748, 0.03691981, 0.04037585, 0.02833889,
        0.02395337]),
 'mean_test_score': array([-0.01365635, -0.01350189, -0.01394416, -0.01411136, -0.01528652,
        -0.01494301]),
 'param_learning_rate': masked_array(data=[0.005, 0.01, 0.02, 0.05, 0.08, 0.1],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'learning_rate': 0.005},
  {'learning_rate': 0.01},
  {'learning_rate': 0.02},
  {'learning_rate': 0.05},
  {'learning_rate': 0.08},
  {'learning_rate': 0.1}],
 'rank_test_score': array([2, 1, 3, 4, 6, 5], dtype=int32),
 'split0_test_score': array([-0.0153815 , -0.01531004, -0.01556661, -0.01681207, -0.01777331,
        -0.0168177 ]),
 'split1_test_score': array([-0.00952055, -0.00913283, -0.0093937 , -0.0099533 , -0.00982569,
        -0.01049724]),
 

#### 7、模型评估

In [16]:
params = {'learning_rate': 0.01,
      'n_estimators': 3000, 
      'max_depth': 5,
      'subsample': 0.6,
      'colsample_bytree': 0.7,
      'min_child_weight': 3,
      'seed': 52,
      'gamma': 0,
      'reg_alpha': 0,
      'reg_lambda': 1}
xgbr = xgb.XGBRegressor(**params)
score = rmse_cv(xgbr)
print('RMSE of XGBoost: {}({})'.format(score.mean(), score.std()))

[07:28:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:28:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:28:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:29:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:29:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:30:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:30:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:30:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

#### 8、在训练集上训练模型并提交预测结果

In [19]:
xgbr_trained = xgbr.fit(np.array(train), np.array(y))
y_preds = xgbr.predict(np.array(train))
model_score = rmse(y, y_preds)
print('RMSE of XGBoost: {}'.format(model_score))

[07:42:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE of XGBoost: 0.02286417135304461


In [22]:
sample_submission = pd.read_csv('sample_submission.csv')
xgbr_preds = xgbr.predict(np.array(test))
xgbr_submission = pd.DataFrame({'Id': sample_submission['Id'], 'SalePrice': np.expm1(xgbr_preds)})
xgbr_submission.to_csv('xgb_optimation_submission.csv', index=False)

### LightGBM调参

#### 1、n_estimators

In [23]:
params = {'learning_rate': 0.01,
      'n_estimators': 3000, 
      'max_depth': 5,
      'num_leaves': 40,
      'subsample': 0.6,
      'colsample_bytree': 0.7,
      'min_child_weight': 0.001,
      'min_child_samples': 21,
      'random_state': 42,
      'reg_alpha': 0,
      'reg_lambda': 1}
cv_params = {'n_estimators': [200, 500, 800, 1000, 2000, 3000, 5000, 6000]}
lgbr = lgb.LGBMRegressor(**params)
grid_lgbr = GridSearchCV(estimator=lgbr, param_grid=cv_params, scoring='neg_mean_squared_error', cv=10, verbose=1)
grid_lgbr.fit(np.array(train), np.array(y))
result = grid_lgbr.cv_results_
#print('每轮迭代运行结果:{}, {}'.format(result['mean_train_score'], result['std_train_score']))
print('参数的最佳取值：{}'.format(grid_lgbr.best_params_))
print('最佳模型得分:{}'.format(grid_lgbr.best_score_))

Fitting 10 folds for each of 8 candidates, totalling 80 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:  3.6min finished


参数的最佳取值：{'n_estimators': 2000}
最佳模型得分:-0.01416341091297445


In [24]:
result

{'mean_fit_time': array([0.36328118, 0.74690835, 1.17250834, 1.32298245, 2.36145921,
        3.42600355, 5.47474439, 6.61960335]),
 'mean_score_time': array([0.00251958, 0.00552309, 0.00920691, 0.01176021, 0.02603095,
        0.03897507, 0.06709406, 0.08671322]),
 'mean_test_score': array([-0.02406443, -0.01547769, -0.01453452, -0.01437802, -0.01416341,
        -0.01427535, -0.01463245, -0.01477547]),
 'param_n_estimators': masked_array(data=[200, 500, 800, 1000, 2000, 3000, 5000, 6000],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_estimators': 200},
  {'n_estimators': 500},
  {'n_estimators': 800},
  {'n_estimators': 1000},
  {'n_estimators': 2000},
  {'n_estimators': 3000},
  {'n_estimators': 5000},
  {'n_estimators': 6000}],
 'rank_test_score': array([8, 7, 4, 3, 1, 2, 5, 6], dtype=int32),
 'split0_test_score': array([-0.0216625 , -0.01532561, -0.01492827, -0.0149614 , -0.01507986,
  

In [25]:
params = {'learning_rate': 0.01,
      'n_estimators': 2000, 
      'max_depth': 5,
      'num_leaves': 40,
      'subsample': 0.6,
      'colsample_bytree': 0.7,
      'min_child_weight': 0.001,
      'min_child_samples': 21,
      'random_state': 42,
      'reg_alpha': 0,
      'reg_lambda': 1}
cv_params = {'n_estimators': [1200, 1500, 1800, 2000, 2200, 2500, 2800]}
lgbr = lgb.LGBMRegressor(**params)
grid_lgbr = GridSearchCV(estimator=lgbr, param_grid=cv_params, scoring='neg_mean_squared_error', cv=10, verbose=1)
grid_lgbr.fit(np.array(train), np.array(y))
result = grid_lgbr.cv_results_
#print('每轮迭代运行结果:{}, {}'.format(result['mean_train_score'], result['std_train_score']))
print('参数的最佳取值：{}'.format(grid_lgbr.best_params_))
print('最佳模型得分:{}'.format(grid_lgbr.best_score_))

Fitting 10 folds for each of 7 candidates, totalling 70 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  70 out of  70 | elapsed:  2.9min finished


参数的最佳取值：{'n_estimators': 1800}
最佳模型得分:-0.014137903179556275


In [26]:
params = {'learning_rate': 0.01,
      'n_estimators': 1800, 
      'max_depth': 5,
      'num_leaves': 40,
      'subsample': 0.6,
      'colsample_bytree': 0.7,
      'min_child_weight': 0.001,
      'min_child_samples': 21,
      'random_state': 42,
      'reg_alpha': 0,
      'reg_lambda': 1}
cv_params = {'n_estimators': [1600, 1650, 1700, 1750, 1800, 1850, 1900, 1950]}
lgbr = lgb.LGBMRegressor(**params)
grid_lgbr = GridSearchCV(estimator=lgbr, param_grid=cv_params, scoring='neg_mean_squared_error', cv=10, verbose=1)
grid_lgbr.fit(np.array(train), np.array(y))
result = grid_lgbr.cv_results_
#print('每轮迭代运行结果:{}, {}'.format(result['mean_train_score'], result['std_train_score']))
print('参数的最佳取值：{}'.format(grid_lgbr.best_params_))
print('最佳模型得分:{}'.format(grid_lgbr.best_score_))

Fitting 10 folds for each of 8 candidates, totalling 80 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:  3.0min finished


参数的最佳取值：{'n_estimators': 1850}
最佳模型得分:-0.014133655367300883


#### 2、max_depth和num_leaves

In [27]:
params = {'learning_rate': 0.01,
      'n_estimators': 1850, 
      'max_depth': 5,
      'num_leaves': 40,
      'subsample': 0.6,
      'colsample_bytree': 0.7,
      'min_child_weight': 0.001,
      'min_child_samples': 21,
      'random_state': 42,
      'reg_alpha': 0,
      'reg_lambda': 1}
cv_params = {'max_depth': [4, 5, 6, 7, 8, 9, 10], 'num_leaves': [10, 20, 30, 40]}
lgbr = lgb.LGBMRegressor(**params)
grid_lgbr = GridSearchCV(estimator=lgbr, param_grid=cv_params, scoring='neg_mean_squared_error', cv=10, verbose=1)
grid_lgbr.fit(np.array(train), np.array(y))
result = grid_lgbr.cv_results_
#print('每轮迭代运行结果:{}, {}'.format(result['mean_train_score'], result['std_train_score']))
print('参数的最佳取值：{}'.format(grid_lgbr.best_params_))
print('最佳模型得分:{}'.format(grid_lgbr.best_score_))

Fitting 10 folds for each of 28 candidates, totalling 280 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 280 out of 280 | elapsed: 13.4min finished


参数的最佳取值：{'max_depth': 4, 'num_leaves': 20}
最佳模型得分:-0.013875783066601685


In [28]:
result

{'mean_fit_time': array([1.73396497, 1.79216259, 1.78766384, 1.79327052, 2.85061455,
        2.30685484, 2.25096986, 2.30711894, 2.15980508, 2.75754461,
        2.74899204, 2.67503693, 2.24028909, 3.05165689, 3.11824713,
        3.10729084, 2.90240254, 3.45427744, 3.61876843, 3.51873364,
        2.22276514, 3.4164    , 3.75529287, 3.81231632, 2.21431904,
        3.57637467, 4.05682294, 4.11013718]),
 'mean_score_time': array([0.02262681, 0.02307589, 0.02305231, 0.02279124, 0.03460968,
        0.02893906, 0.02885568, 0.02880771, 0.02818291, 0.03521419,
        0.03435841, 0.03349514, 0.02857547, 0.03863497, 0.03853676,
        0.03904788, 0.03471916, 0.04216104, 0.04475651, 0.04381151,
        0.02774668, 0.04179821, 0.04536602, 0.04657865, 0.02812948,
        0.04291713, 0.04761159, 0.04919436]),
 'mean_test_score': array([-0.01409869, -0.01387578, -0.01387578, -0.01387578, -0.01406463,
        -0.01417941, -0.0141109 , -0.01413366, -0.01407212, -0.01429851,
        -0.01446278, -0.014

#### 3、subsample和colsample_bytree

In [30]:
params = {'learning_rate': 0.01,
      'n_estimators': 1850, 
      'max_depth': 4,
      'num_leaves': 20,
      'subsample': 0.6,
      'colsample_bytree': 0.7,
      'min_child_weight': 0.001,
      'min_child_samples': 21,
      'random_state': 42,
      'reg_alpha': 0,
      'reg_lambda': 1}
cv_params = {'subsample': [0.6, 0.7, 0.8, 0.9, 1.0], 'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0]}
lgbr = lgb.LGBMRegressor(**params)
grid_lgbr = GridSearchCV(estimator=lgbr, param_grid=cv_params, scoring='neg_mean_squared_error', cv=10, verbose=1)
grid_lgbr.fit(np.array(train), np.array(y))
result = grid_lgbr.cv_results_
#print('每轮迭代运行结果:{}, {}'.format(result['mean_train_score'], result['std_train_score']))
print('参数的最佳取值：{}'.format(grid_lgbr.best_params_))
print('最佳模型得分:{}'.format(grid_lgbr.best_score_))

Fitting 10 folds for each of 25 candidates, totalling 250 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:  8.4min finished


参数的最佳取值：{'colsample_bytree': 0.6, 'subsample': 0.6}
最佳模型得分:-0.013818279893477114


In [31]:
result

{'mean_fit_time': array([1.64396176, 1.61300182, 1.61825509, 1.60039117, 1.73362584,
        1.79676082, 1.76831002, 1.77585108, 1.76979437, 1.77159867,
        1.95731888, 1.95850058, 1.96509817, 1.97931077, 1.95889766,
        2.15079374, 2.2094703 , 2.12838409, 2.15558629, 2.16080089,
        2.47246084, 2.32840312, 2.35173106, 2.38614638, 2.36009274]),
 'mean_score_time': array([0.02441113, 0.02384112, 0.02402968, 0.02366283, 0.02950702,
        0.02416947, 0.02359538, 0.02436526, 0.02381809, 0.02408462,
        0.02464352, 0.02377119, 0.02477858, 0.02418201, 0.02440002,
        0.02469022, 0.02437212, 0.0241565 , 0.02365475, 0.02412951,
        0.02569928, 0.02338061, 0.02508268, 0.02431736, 0.02386799]),
 'mean_test_score': array([-0.01381828, -0.01381828, -0.01381828, -0.01381828, -0.01381828,
        -0.01387578, -0.01387578, -0.01387578, -0.01387578, -0.01387578,
        -0.01419723, -0.01419723, -0.01419723, -0.01419723, -0.01419723,
        -0.01439637, -0.01439637, -0.01439

#### 4、min_child_weight和min_child_samples

In [32]:
params = {'learning_rate': 0.01,
      'n_estimators': 1850, 
      'max_depth': 4,
      'num_leaves': 20,
      'subsample': 0.6,
      'colsample_bytree': 0.6,
      'min_child_weight': 0.001,
      'min_child_samples': 21,
      'random_state': 42,
      'reg_alpha': 0,
      'reg_lambda': 1}
cv_params = {'min_child_weight': [0.001, 0.002, 0.005, 0.008, 0.01, 0.02, 0.05], 'min_child_samples': [5, 10, 15, 20, 21, 25]}
lgbr = lgb.LGBMRegressor(**params)
grid_lgbr = GridSearchCV(estimator=lgbr, param_grid=cv_params, scoring='neg_mean_squared_error', cv=10, verbose=1)
grid_lgbr.fit(np.array(train), np.array(y))
result = grid_lgbr.cv_results_
#print('每轮迭代运行结果:{}, {}'.format(result['mean_train_score'], result['std_train_score']))
print('参数的最佳取值：{}'.format(grid_lgbr.best_params_))
print('最佳模型得分:{}'.format(grid_lgbr.best_score_))

Fitting 10 folds for each of 42 candidates, totalling 420 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 420 out of 420 | elapsed: 12.3min finished


参数的最佳取值：{'min_child_samples': 21, 'min_child_weight': 0.001}
最佳模型得分:-0.013818279893477114


In [33]:
result

{'mean_fit_time': array([1.9689527 , 1.95498285, 1.95396619, 1.95819476, 2.02154586,
        1.97477007, 1.96248043, 1.81906269, 1.81380773, 1.81392717,
        1.922031  , 1.91738238, 1.82323477, 1.82308898, 1.69869943,
        1.73395798, 1.72690392, 1.7759913 , 1.78114302, 1.78387098,
        1.74740906, 1.79377067, 1.6225359 , 1.60954332, 1.60326838,
        1.609126  , 1.60696466, 1.60342436, 1.59300976, 1.67651534,
        1.60292559, 1.58916082, 1.59629402, 1.6079495 , 1.60067475,
        1.53513186, 1.56054807, 1.55584438, 1.54361894, 1.54104311,
        1.70833974, 1.54863577]),
 'mean_score_time': array([0.02445314, 0.02432151, 0.02422614, 0.02452335, 0.02471414,
        0.02413599, 0.02422316, 0.02422619, 0.02445507, 0.02458189,
        0.02527494, 0.02539561, 0.02404635, 0.02465789, 0.02464151,
        0.02465591, 0.02506652, 0.02513893, 0.02460186, 0.02656541,
        0.02501915, 0.02696495, 0.02458901, 0.02431498, 0.02436025,
        0.02405581, 0.02422616, 0.02444775, 0.

#### 5、reg_alpha和reg_lambda

In [35]:
params = {'learning_rate': 0.01,
      'n_estimators': 1850, 
      'max_depth': 4,
      'num_leaves': 20,
      'subsample': 0.6,
      'colsample_bytree': 0.6,
      'min_child_weight': 0.001,
      'min_child_samples': 21,
      'random_state': 42,
      'reg_alpha': 0,
      'reg_lambda': 1}
cv_params = {'reg_alpha': [0, 0.05, 0.1, 0.5, 1, 2, 3], 'reg_lambda': [0.05, 0.1, 0.5, 1, 2, 3]}
lgbr = lgb.LGBMRegressor(**params)
grid_lgbr = GridSearchCV(estimator=lgbr, param_grid=cv_params, scoring='neg_mean_squared_error', cv=10, verbose=1)
grid_lgbr.fit(np.array(train), np.array(y))
result = grid_lgbr.cv_results_
#print('每轮迭代运行结果:{}, {}'.format(result['mean_train_score'], result['std_train_score']))
print('参数的最佳取值：{}'.format(grid_lgbr.best_params_))
print('最佳模型得分:{}'.format(grid_lgbr.best_score_))

Fitting 10 folds for each of 42 candidates, totalling 420 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 420 out of 420 | elapsed: 11.1min finished


参数的最佳取值：{'reg_alpha': 0, 'reg_lambda': 0.05}
最佳模型得分:-0.013749403416544945


In [36]:
result

{'mean_fit_time': array([1.60859127, 1.61338716, 1.60780816, 1.61030126, 1.60379112,
        1.60750716, 1.73709533, 1.66381371, 1.66974664, 1.65426681,
        1.63110368, 1.61240075, 1.66580362, 1.67241127, 1.66642861,
        1.69009402, 1.79282868, 1.6476521 , 1.68173707, 1.70654979,
        1.70440319, 1.6709168 , 1.71427481, 1.64946387, 1.70986683,
        1.65124054, 1.63689225, 1.6239141 , 1.65973372, 1.6339812 ,
        1.42474692, 1.39159753, 1.39166746, 1.40600307, 1.39640951,
        1.46104305, 1.1930007 , 1.1818526 , 1.16700189, 1.19942396,
        1.18900833, 1.21089697]),
 'mean_score_time': array([0.02449822, 0.02509604, 0.02481582, 0.02446148, 0.02442393,
        0.02439198, 0.02581816, 0.02522902, 0.02544799, 0.02483044,
        0.02589951, 0.02522395, 0.0260618 , 0.02442043, 0.02490325,
        0.02516918, 0.02645836, 0.02480161, 0.02537348, 0.02605722,
        0.02639475, 0.02529082, 0.02680974, 0.02574973, 0.02864285,
        0.0255151 , 0.02489049, 0.0248965 , 0.

#### 6、模型评估

In [37]:
params = {'learning_rate': 0.01,
      'n_estimators': 1850, 
      'max_depth': 4,
      'num_leaves': 20,
      'subsample': 0.6,
      'colsample_bytree': 0.6,
      'min_child_weight': 0.001,
      'min_child_samples': 21,
      'random_state': 42,
      'reg_alpha': 0,
      'reg_lambda': 0.05}
lgbr = lgb.LGBMRegressor(**params)
score = rmse_cv(lgbr)
print('RMSE of LightGBM: {}({})'.format(score.mean(), score.std()))

RMSE of LightGBM: 0.118318792967013(0.010364808635531306)


#### 7、在训练集上训练并提交预测结果

In [38]:
lgbr_trained = lgbr.fit(np.array(train), np.array(y))
y_preds = lgbr.predict(np.array(train))
model_score = rmse(y, y_preds)
print('RMSE of LightGBM: {}'.format(model_score))

RMSE of LightGBM: 0.0641626832182992


In [39]:
lgbr_preds = lgbr.predict(np.array(test))
lgbr_submission = pd.DataFrame({'Id': sample_submission['Id'], 'SalePrice': np.expm1(lgbr_preds)})
lgbr_submission.to_csv('lgb_optimation_submission.csv', index=False)

### Gradient Boosting调参

#### 1、n_estimators

In [41]:
params = {'loss': 'huber',
      'criterion': 'mse', 
      'learning_rate': 0.1,
      'n_estimators': 300, 
      'max_depth': 6,
      'subsample': 0.6,
      'min_samples_split': 5,
      'min_samples_leaf': 5,
      'max_features': 0.6,
      'random_state': 32,
      'alpha': 0.5}
cv_params = {'n_estimators': [80, 100, 200, 300, 400, 500, 600, 800]}
gbr = GradientBoostingRegressor(**params)
grid_gbr = GridSearchCV(estimator=gbr, param_grid=cv_params, scoring='neg_mean_squared_error', cv=10, verbose=1)
grid_gbr.fit(np.array(train), np.array(y))
result = grid_gbr.cv_results_
#print('每轮迭代运行结果:{}, {}'.format(result['mean_train_score'], result['std_train_score']))
print('参数的最佳取值：{}'.format(grid_gbr.best_params_))
print('最佳模型得分:{}'.format(grid_gbr.best_score_))

Fitting 10 folds for each of 8 candidates, totalling 80 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:  5.9min finished


参数的最佳取值：{'n_estimators': 600}
最佳模型得分:-0.014977598979614168


In [42]:
params = {'loss': 'huber',
      'criterion': 'mse', 
      'learning_rate': 0.1,
      'n_estimators': 600, 
      'max_depth': 6,
      'subsample': 0.6,
      'min_samples_split': 5,
      'min_samples_leaf': 5,
      'max_features': 0.6,
      'random_state': 32,
      'alpha': 0.5}
cv_params = {'n_estimators': [550, 600, 650, 700, 750]}
gbr = GradientBoostingRegressor(**params)
grid_gbr = GridSearchCV(estimator=gbr, param_grid=cv_params, scoring='neg_mean_squared_error', cv=10, verbose=1)
grid_gbr.fit(np.array(train), np.array(y))
result = grid_gbr.cv_results_
#print('每轮迭代运行结果:{}, {}'.format(result['mean_train_score'], result['std_train_score']))
print('参数的最佳取值：{}'.format(grid_gbr.best_params_))
print('最佳模型得分:{}'.format(grid_gbr.best_score_))

Fitting 10 folds for each of 5 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  6.3min finished


参数的最佳取值：{'n_estimators': 600}
最佳模型得分:-0.014977598979614168


In [43]:
result

{'mean_fit_time': array([6.40975561, 7.02159927, 7.5259428 , 8.44443698, 8.6572866 ]),
 'mean_score_time': array([0.00456064, 0.00480833, 0.00509157, 0.00554054, 0.00577326]),
 'mean_test_score': array([-0.01498572, -0.0149776 , -0.01498595, -0.01499545, -0.01500083]),
 'param_n_estimators': masked_array(data=[550, 600, 650, 700, 750],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_estimators': 550},
  {'n_estimators': 600},
  {'n_estimators': 650},
  {'n_estimators': 700},
  {'n_estimators': 750}],
 'rank_test_score': array([2, 1, 3, 4, 5], dtype=int32),
 'split0_test_score': array([-0.01573263, -0.01573707, -0.01579655, -0.01580227, -0.01581406]),
 'split1_test_score': array([-0.01196664, -0.01191671, -0.01192112, -0.0119674 , -0.01196313]),
 'split2_test_score': array([-0.01088311, -0.01082731, -0.01073828, -0.01074316, -0.01077002]),
 'split3_test_score': array([-0.01870238, -0.01866833, -0.01869071, -0.01

#### 2、max_depth和min_samples_split

In [44]:
params = {'loss': 'huber',
      'criterion': 'mse', 
      'learning_rate': 0.1,
      'n_estimators': 600, 
      'max_depth': 6,
      'subsample': 0.6,
      'min_samples_split': 5,
      'min_samples_leaf': 5,
      'max_features': 0.6,
      'random_state': 32,
      'alpha': 0.5}
cv_params = {'max_depth': [4, 5, 6, 7, 8, 9, 10], 'min_samples_split': [5, 10, 15, 20, 30]}
gbr = GradientBoostingRegressor(**params)
grid_gbr = GridSearchCV(estimator=gbr, param_grid=cv_params, scoring='neg_mean_squared_error', cv=10, verbose=1)
grid_gbr.fit(np.array(train), np.array(y))
result = grid_gbr.cv_results_
#print('每轮迭代运行结果:{}, {}'.format(result['mean_train_score'], result['std_train_score']))
print('参数的最佳取值：{}'.format(grid_gbr.best_params_))
print('最佳模型得分:{}'.format(grid_gbr.best_score_))

Fitting 10 folds for each of 35 candidates, totalling 350 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 350 out of 350 | elapsed: 41.4min finished


参数的最佳取值：{'max_depth': 4, 'min_samples_split': 20}
最佳模型得分:-0.013549971127499515


In [45]:
result

{'mean_fit_time': array([4.69205875, 4.71579618, 4.58660986, 4.56870308, 4.44460218,
        5.79943779, 5.81757364, 5.63409076, 5.58850024, 5.50859115,
        7.00632105, 6.96717567, 6.59292946, 6.3695153 , 6.26778316,
        8.13781629, 8.25375459, 7.68779619, 7.75606222, 6.9371448 ,
        8.13292999, 8.17464676, 8.65023701, 8.1903275 , 7.79353244,
        7.70653279, 7.86326833, 8.84036481, 8.92861869, 8.48641727,
        7.42953613, 7.43896382, 8.42958648, 9.5050421 , 9.07760022]),
 'mean_score_time': array([0.00349917, 0.00336659, 0.00342677, 0.0035326 , 0.00359862,
        0.00412784, 0.00414326, 0.00415456, 0.00438585, 0.00429909,
        0.00489256, 0.00482271, 0.00471992, 0.00462916, 0.00457733,
        0.00560069, 0.00571196, 0.00558443, 0.00552766, 0.00520067,
        0.00560374, 0.00561574, 0.00610831, 0.00597327, 0.00592606,
        0.0053899 , 0.00546293, 0.00631795, 0.00640659, 0.00640326,
        0.00524132, 0.0052357 , 0.0060138 , 0.00679586, 0.00672843]),
 'mean_t

#### 3、min_samples_leaf

In [46]:
params = {'loss': 'huber',
      'criterion': 'mse', 
      'learning_rate': 0.1,
      'n_estimators': 600, 
      'max_depth': 4,
      'subsample': 0.6,
      'min_samples_split': 20,
      'min_samples_leaf': 5,
      'max_features': 0.6,
      'random_state': 32,
      'alpha': 0.5}
cv_params = {'min_samples_leaf': [5, 10, 15, 20]}
gbr = GradientBoostingRegressor(**params)
grid_gbr = GridSearchCV(estimator=gbr, param_grid=cv_params, scoring='neg_mean_squared_error', cv=10, verbose=1)
grid_gbr.fit(np.array(train), np.array(y))
result = grid_gbr.cv_results_
#print('每轮迭代运行结果:{}, {}'.format(result['mean_train_score'], result['std_train_score']))
print('参数的最佳取值：{}'.format(grid_gbr.best_params_))
print('最佳模型得分:{}'.format(grid_gbr.best_score_))

Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  3.0min finished


参数的最佳取值：{'min_samples_leaf': 5}
最佳模型得分:-0.013549971127499515


In [47]:
result

{'mean_fit_time': array([4.47414668, 4.53486431, 4.4072844 , 4.32847376]),
 'mean_score_time': array([0.00344181, 0.00361266, 0.0035955 , 0.00360644]),
 'mean_test_score': array([-0.01354997, -0.0141878 , -0.01371437, -0.01441598]),
 'param_min_samples_leaf': masked_array(data=[5, 10, 15, 20],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'min_samples_leaf': 5},
  {'min_samples_leaf': 10},
  {'min_samples_leaf': 15},
  {'min_samples_leaf': 20}],
 'rank_test_score': array([1, 3, 2, 4], dtype=int32),
 'split0_test_score': array([-0.01380506, -0.01496863, -0.01375436, -0.01428024]),
 'split1_test_score': array([-0.00848648, -0.01014579, -0.00891929, -0.00973996]),
 'split2_test_score': array([-0.01104128, -0.01169375, -0.01161256, -0.01210855]),
 'split3_test_score': array([-0.01920902, -0.01982954, -0.0196035 , -0.02027373]),
 'split4_test_score': array([-0.0204864 , -0.02099082, -0.01935932, -0.02013519]),
 'split5_test

#### 4、max_features

In [48]:
params = {'loss': 'huber',
      'criterion': 'mse', 
      'learning_rate': 0.1,
      'n_estimators': 600, 
      'max_depth': 4,
      'subsample': 0.6,
      'min_samples_split': 20,
      'min_samples_leaf': 5,
      'max_features': 0.6,
      'random_state': 32,
      'alpha': 0.5}
cv_params = {'max_features': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}
gbr = GradientBoostingRegressor(**params)
grid_gbr = GridSearchCV(estimator=gbr, param_grid=cv_params, scoring='neg_mean_squared_error', cv=10, verbose=1)
grid_gbr.fit(np.array(train), np.array(y))
result = grid_gbr.cv_results_
#print('每轮迭代运行结果:{}, {}'.format(result['mean_train_score'], result['std_train_score']))
print('参数的最佳取值：{}'.format(grid_gbr.best_params_))
print('最佳模型得分:{}'.format(grid_gbr.best_score_))

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:  5.2min finished


参数的最佳取值：{'max_features': 0.6}
最佳模型得分:-0.013549971127499515


In [49]:
result

{'mean_fit_time': array([4.0276861 , 4.48107996, 4.99831038, 5.39778967, 6.00730252,
        6.28444765]),
 'mean_score_time': array([0.00360675, 0.00338058, 0.00338416, 0.00342782, 0.00344207,
        0.00345123]),
 'mean_test_score': array([-0.01398766, -0.01354997, -0.01413735, -0.01412094, -0.01393782,
        -0.01441619]),
 'param_max_features': masked_array(data=[0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_features': 0.5},
  {'max_features': 0.6},
  {'max_features': 0.7},
  {'max_features': 0.8},
  {'max_features': 0.9},
  {'max_features': 1.0}],
 'rank_test_score': array([3, 1, 5, 4, 2, 6], dtype=int32),
 'split0_test_score': array([-0.01533972, -0.01380506, -0.01477462, -0.01454269, -0.01446978,
        -0.0154407 ]),
 'split1_test_score': array([-0.00986073, -0.00848648, -0.00944819, -0.00959854, -0.00943181,
        -0.00921154]),
 'split2_test_score': arra

#### 5、subsample

In [50]:
params = {'loss': 'huber',
      'criterion': 'mse', 
      'learning_rate': 0.1,
      'n_estimators': 600, 
      'max_depth': 4,
      'subsample': 0.6,
      'min_samples_split': 20,
      'min_samples_leaf': 5,
      'max_features': 0.6,
      'random_state': 32,
      'alpha': 0.5}
cv_params = {'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}
gbr = GradientBoostingRegressor(**params)
grid_gbr = GridSearchCV(estimator=gbr, param_grid=cv_params, scoring='neg_mean_squared_error', cv=10, verbose=1)
grid_gbr.fit(np.array(train), np.array(y))
result = grid_gbr.cv_results_
#print('每轮迭代运行结果:{}, {}'.format(result['mean_train_score'], result['std_train_score']))
print('参数的最佳取值：{}'.format(grid_gbr.best_params_))
print('最佳模型得分:{}'.format(grid_gbr.best_score_))

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:  5.2min finished


参数的最佳取值：{'subsample': 0.6}
最佳模型得分:-0.013549971127499515


#### 6、alpha

In [10]:
params = {'loss': 'huber',
      'criterion': 'mse', 
      'learning_rate': 0.1,
      'n_estimators': 600, 
      'max_depth': 4,
      'subsample': 0.6,
      'min_samples_split': 20,
      'min_samples_leaf': 5,
      'max_features': 0.6,
      'random_state': 32,
      'alpha': 0.5}
cv_params = {'alpha': [0, 0.2, 0.5, 0.8, 1, 2, 3]}
gbr = GradientBoostingRegressor(**params)
grid_gbr = GridSearchCV(estimator=gbr, param_grid=cv_params, scoring='neg_mean_squared_error', cv=10, verbose=1)
grid_gbr.fit(np.array(train), np.array(y))
result = grid_gbr.cv_results_
#print('每轮迭代运行结果:{}, {}'.format(result['mean_train_score'], result['std_train_score']))
print('参数的最佳取值：{}'.format(grid_gbr.best_params_))
print('最佳模型得分:{}'.format(grid_gbr.best_score_))

Fitting 10 folds for each of 7 candidates, totalling 70 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  70 out of  70 | elapsed:  2.1min finished


参数的最佳取值：{'alpha': 0.5}
最佳模型得分:-0.013549971127499515


#### 7、模型评估

In [11]:
params = {'loss': 'huber',
      'criterion': 'mse', 
      'learning_rate': 0.1,
      'n_estimators': 600, 
      'max_depth': 4,
      'subsample': 0.6,
      'min_samples_split': 20,
      'min_samples_leaf': 5,
      'max_features': 0.6,
      'random_state': 32,
      'alpha': 0.5}
gbr = GradientBoostingRegressor(**params)
score = rmse_cv(gbr)
print('RMSE of Gradient Boosting: {}({})'.format(score.mean(), score.std()))

RMSE of Gradient Boosting: 0.11865538871859024(0.009497984510023386)


In [12]:
gbr_trained = gbr.fit(np.array(train), np.array(y))
y_preds = gbr.predict(np.array(train))
model_score = rmse(y, y_preds)
print('RMSE of Gradient Boosting: {}'.format(model_score))

RMSE of Gradient Boosting: 0.06583064311955181


In [13]:
sample_submission = pd.read_csv('sample_submission.csv')
gbr_preds = gbr.predict(np.array(test))
gbr_submission = pd.DataFrame({'Id': sample_submission['Id'], 'SalePrice': np.expm1(gbr_preds)})
gbr_submission.to_csv('gbr_optimation_submission.csv', index=False)